# Домашнее задание по занатию 15

Задание: 
* Решить задачу с применением линейной регрессии. 
* Объяснить выбор функции ошибок и метрики. 
* Дать субъективную оценку качества, глядя на значения, полученные при помощи функции ошибок и метрик.

Цель:
* Применить модель линейной регрессии для предсказания результатов.
* Расчитать функции ошибок.

## Импорт библиотек и загрузка данных

In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import pandas as pd

In [42]:
data = load_iris()
print(f'Feature values:\n{data.data[:5]}')
print(f'Feature names: {data.feature_names}')

data.target = np.array([0 if i==0 else 1 for i in data.target])
print(f'Target: {data.target}')

np.unique(data.target, return_counts=True)

Feature values:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
Feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Target: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]


(array([0, 1]), array([ 50, 100], dtype=int64))

In [43]:
# разделим данные на тренирововчный и тестовый датасеты
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, train_size=0.55,random_state=42)

## Первичный анализ данных

In [44]:
df = pd.DataFrame(data= np.c_[X_train, y_train],columns= data['feature_names'] + ['target'])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,4.6,3.2,1.4,0.2,0.0
1,6.2,2.9,4.3,1.3,1.0
2,5.7,2.5,5.0,2.0,1.0
3,5.5,4.2,1.4,0.2,0.0
4,6.0,3.0,4.8,1.8,1.0


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  82 non-null     float64
 1   sepal width (cm)   82 non-null     float64
 2   petal length (cm)  82 non-null     float64
 3   petal width (cm)   82 non-null     float64
 4   target             82 non-null     float64
dtypes: float64(5)
memory usage: 3.3 KB


In [46]:
# посмотрим корреляцию между фичами и таргетом
df.corr()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
sepal length (cm),1.000000,-0.056297,0.851571,0.775531,0.698696
sepal width (cm),-0.056297,1.000000,-0.368543,-0.287466,-0.570445
petal length (cm),0.851571,-0.368543,1.000000,0.955804,0.919304
petal width (cm),0.775531,-0.287466,0.955804,1.000000,0.871017
target,0.698696,-0.570445,0.919304,0.871017,1.000000


In [47]:
df["sepal_s"] = df["sepal length (cm)"]*df["sepal width (cm)"]
df["petal_s"] = df["petal length (cm)"]*df["petal width (cm)"]
df.corr()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,sepal_s,petal_s
sepal length (cm),1.000000,-0.056297,0.851571,0.775531,0.698696,0.711226,0.813627
sepal width (cm),-0.056297,1.000000,-0.368543,-0.287466,-0.570445,0.658665,-0.204764
petal length (cm),0.851571,-0.368543,1.000000,0.955804,0.919304,0.388444,0.949727
petal width (cm),0.775531,-0.287466,0.955804,1.000000,0.871017,0.395362,0.983128
target,0.698696,-0.570445,0.919304,0.871017,1.000000,0.134292,0.802636
sepal_s,0.711226,0.658665,0.388444,0.395362,0.134292,1.000000,0.480973
petal_s,0.813627,-0.204764,0.949727,0.983128,0.802636,0.480973,1.000000


##### Вывод

Так как существует большая корреляция меджу показателем petal length и petal width и sepal length для можеди решила использовать показатели площади:
* sepal length * sepal width
* petal length * petal width

## Обучение модели

In [48]:
# Подготавливаем данные
X_train = np.append(X_train, [[i*t] for i, t in zip(X_train[:,0], X_train[:,1])], axis=1)
X_train = np.append(X_train, [[i*t] for i, t in zip(X_train[:,2], X_train[:,3])], axis=1)

In [49]:
# обучаем модель
lreg = LinearRegression()
lreg.fit(X_train[:, 4:], y_train)

print(f'Коэффициенты модели: {lreg.coef_}')
print(f'Коэффициент детерминации на тренировочных данных: {lreg.score(X_train[:, 4:], y_train)}')

Коэффициенты модели: [-0.04552092  0.09570242]
Коэффициент детерминации на тренировочных данных: 0.7266797312445694


In [50]:
# Рассчитываем прогноз на тренировочных данных
y_pred_train = lreg.predict(X_train[:, 4:])

print(f'RMSE: {metrics.mean_squared_error(y_train, y_pred_train, squared=False)}')
print(f'MSE: {metrics.mean_squared_error(y_train, y_pred_train, squared=True)}')

RMSE: 0.23486158691049355
MSE: 0.055159965006115315


In [51]:
# Переводим в классификатоор
y_pred_traint_class = list(map(lambda i: 0 if i < 0.5 else 1, y_pred_train))

print(f'Accuracy: {metrics.accuracy_score(y_train, y_pred_traint_class)}')

Accuracy: 1.0


## Предсказание

In [52]:
# Подготавливаем данные
X_test = np.append(X_test, [[i*t] for i, t in zip(X_test[:,0], X_test[:,1])], axis=1)
X_test = np.append(X_test, [[i*t] for i, t in zip(X_test[:,2], X_test[:,3])], axis=1)

In [53]:
# Рассчитываем прогноз на тестовых данных
y_pred = lreg.predict(X_test[:, 4:])

print(f'RMSE: {metrics.mean_squared_error(y_test, y_pred, squared=False)}')
print(f'MSE: {metrics.mean_squared_error(y_test, y_pred, squared=True)}')

RMSE: 0.23880500535302357
MSE: 0.05702783058165761


In [54]:
# Переводим в классификатоор
y_pred_class = list(map(lambda i: 0 if i < 0.5 else 1, y_pred))

print(f'Accuracy: {metrics.accuracy_score(y_test, y_pred_class)}')

Accuracy: 1.0
